# <center>Welcome FSP solved with local search based metaheuristics</center>

This notebook presents a practical approach to solving the flowshop problem by implementing well-known local search based metaheuristics. These metaheuristics are effective in generating high-quality solutions for large instances of the problem, requiring only a reasonable amount of computational resources. Compared to heuristics, local search metaheuristics are more effective for solving the flowshop problem because they can escape from local optima and find better solutions

# Table of Contents

1. [Data utils](#Data-utils)
2. [Neighborhood generation](#Neighborhood-generation)
3. [Local Based Metaheuristics](#Local-Based-Metaheuristics)
4. [Tests](#Tests)
<!-- 2. 1. [SWAP](#SWAP)
4. [Random SWAP](#Random-SWAP)
5. [Best SWAP](#Best-SWAP)
6. [First Admissible SWAP](#First-Admissible-SWAP)
7. [First and Best Admissible SWAP](#First-and-Best-Admissible-SWAP) -->

## Data utils

In [2]:
import numpy as np
import random
import time
import math
import pandas as pd
import matplotlib.pyplot as plt
from utils.benchmarks import benchmarks, upper_bound
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

### Path Cost calculation function :
Used to calculate the cost of current node, which is the correct cost starting for the actual path of executed jobs

In [3]:
def calculate_makespan(processing_times, sequence):
    n_jobs = len(sequence)
    n_machines = len(processing_times[0])
    end_time = [[0] * (n_machines + 1) for _ in range(n_jobs + 1)]

    for j in range(1, n_jobs + 1):
        for m in range(1, n_machines + 1):
            end_time[j][m] = max(end_time[j][m - 1], end_time[j - 1]
                                 [m]) + processing_times[sequence[j - 1]][m - 1]

    return end_time[n_jobs][n_machines]

### Random data for tests

In [4]:
rnd_data = np.random.randint(size=(20,5), low=5, high=120)
permutation = np.random.permutation(20).tolist()
print(rnd_data, "\n")
print('Initial solution:', permutation, "\n")

Cmax = calculate_makespan(rnd_data, permutation)
print(f'Makespan: {Cmax}')

[[ 29  66 107  12 118]
 [ 43 103  54  54  47]
 [ 13 117 103  87  87]
 [ 16 106  57 109  60]
 [ 68  93  56  71  24]
 [  8  13 101  38  99]
 [ 34   9  59  90  50]
 [ 93  46  88  69  98]
 [  9  12 115  93  84]
 [ 66  86 102  99  82]
 [ 59  65  53  93  25]
 [ 23  48  49 113  43]
 [ 91 106 117 116 115]
 [  6 102 119  78  30]
 [ 41  28  17  30 111]
 [ 98  75  86 115  81]
 [ 70  19 116  54 106]
 [ 45   9  27  83  97]
 [105  87   7  38  37]
 [ 22  29  33  39  40]] 

Initial solution: [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12] 

Makespan: 2054


### Gantt graph generator

In [5]:
def generate_gantt_chart(processing_times, seq, interval=50, labeled=True):
    data = processing_times.T
    nb_jobs, nb_machines = processing_times.shape
    schedules = np.zeros((nb_machines, nb_jobs), dtype=dict)
    # schedule first job alone first
    task = {"name": "job_{}".format(
        seq[0]+1), "start_time": 0, "end_time": data[0][seq[0]]}

    schedules[0][0] = task
    for m_id in range(1, nb_machines):
        start_t = schedules[m_id-1][0]["end_time"]
        end_t = start_t + data[m_id][0]
        task = {"name": "job_{}".format(
            seq[0]+1), "start_time": start_t, "end_time": end_t}
        schedules[m_id][0] = task

    for index, job_id in enumerate(seq[1::]):
        start_t = schedules[0][index]["end_time"]
        end_t = start_t + data[0][job_id]
        task = {"name": "job_{}".format(
            job_id+1), "start_time": start_t, "end_time": end_t}
        schedules[0][index+1] = task
        for m_id in range(1, nb_machines):
            start_t = max(schedules[m_id][index]["end_time"],
                          schedules[m_id-1][index+1]["end_time"])
            end_t = start_t + data[m_id][job_id]
            task = {"name": "job_{}".format(
                job_id+1), "start_time": start_t, "end_time": end_t}
            schedules[m_id][index+1] = task

    # create a new figure
    fig, ax = plt.subplots(figsize=(18, 8))

    # set y-axis ticks and labels
    y_ticks = list(range(len(schedules)))
    y_labels = [f'Machine {i+1}' for i in y_ticks]
    ax.set_yticks(y_ticks)
    ax.set_yticklabels(y_labels)

    # calculate the total time
    total_time = max([job['end_time'] for proc in schedules for job in proc])

    # set x-axis limits and ticks
    ax.set_xlim(0, total_time)
    x_ticks = list(range(0, total_time+1, interval))
    ax.set_xticks(x_ticks)

    # set grid lines
    ax.grid(True, axis='x', linestyle='--')

    # create a color dictionary to map each job to a color
    color_dict = {}
    for proc in schedules:
        for job in proc:
            if job['name'] not in color_dict:
                color_dict[job['name']] = (np.random.uniform(
                    0, 1), np.random.uniform(0, 1), np.random.uniform(0, 1))

    # plot the bars for each job on each processor
    for i, proc in enumerate(schedules):
        for job in proc:
            start = job['start_time']
            end = job['end_time']
            duration = end - start
            color = color_dict[job['name']]
            ax.barh(i, duration, left=start, height=0.5,
                    align='center', color=color, alpha=0.8)
            if labeled:
                # add job labels
                label_x = start + duration/2
                label_y = i
                ax.text(
                    label_x, label_y, job['name'][4:], ha='center', va='center', fontsize=10)

    plt.show()

### INITIAL SOLUTION METHODS

In [6]:
from utils.initial_sol_methods import neh_algorithm
from utils.initial_sol_methods import PRSKE
from utils.initial_sol_methods import ham_heuristic

## Neighborhood generation

### SWAP

In [7]:
def swap(solution, i, k):
    sol = solution.copy()
    sol[i], sol[k] = sol[k], sol[i]
    return sol

### Random SWAP

In [8]:
def random_swap(solution, processing_times):
    i = random.choice(list(solution))
    j = random.choice(list(solution))

    while i == j:
        j = random.choice(list(solution))

    new_solution = swap(solution, i, j)
    
    return new_solution, calculate_makespan(processing_times, new_solution)

In [9]:
neighbor_sol, neighbor_cmax = random_swap(permutation, rnd_data)

print("Original_solution: ", permutation)
print("Cmax = ", Cmax)

print("\nNeighbor_solution: ", neighbor_sol)
print("Neighor_cmax = ", neighbor_cmax)

Original_solution:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]
Cmax =  2054

Neighbor_solution:  [0, 14, 18, 4, 16, 10, 2, 5, 19, 9, 8, 15, 13, 11, 6, 7, 1, 17, 3, 12]
Neighor_cmax =  2069


### Best SWAP

In [10]:
def best_swap(solution, processing_times):
    sequence = solution.copy()
    num_jobs = len(solution)
    Cmax = calculate_makespan(processing_times, solution)

    for i in range(num_jobs):
        for j in range(i+1, num_jobs):
            new_solution = swap(sequence, i, j)
            makespan = calculate_makespan(processing_times, new_solution)

            if makespan < Cmax:
                sequence = new_solution
                Cmax = makespan

    return sequence, Cmax

In [11]:
neighbor_sol, neighbor_cmax = best_swap(permutation, rnd_data)

print("Original_solution: ", permutation)
print("Cmax = ", Cmax)

print("\nNeighbor_solution: ", neighbor_sol)
print("Neighor_cmax = ", neighbor_cmax)

Original_solution:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]
Cmax =  2054

Neighbor_solution:  [3, 8, 17, 12, 0, 9, 15, 18, 19, 4, 16, 14, 13, 11, 5, 7, 1, 2, 6, 10]
Neighor_cmax =  1779


### First Admissible SWAP

In [12]:
def first_admissible_swap(solution, processing_times):

    num_jobs = len(solution)
    Cmax = calculate_makespan(processing_times, solution)

    for i in range(num_jobs):
        for j in range(i+1, num_jobs):
            new_solution = swap(solution, i, j)
            makespan = calculate_makespan(processing_times, new_solution)

            if makespan < Cmax:
                return new_solution, makespan

    return solution, Cmax 

In [13]:
neighbor_sol, neighbor_cmax = first_admissible_swap(permutation, rnd_data)

print("Original_solution: ", permutation)
print("Cmax = ", Cmax)

print("\nNeighbor_solution: ", neighbor_sol)
print("Neighor_cmax = ", neighbor_cmax)

Original_solution:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]
Cmax =  2054

Neighbor_solution:  [2, 15, 18, 4, 16, 10, 0, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]
Neighor_cmax =  2042


### First and Best Admissible SWAP

In [14]:
def fba_swap(solution, processing_times, best_global_sol):
    sequence = solution.copy()
    num_jobs = len(sequence)
    Cmax = calculate_makespan(processing_times, sequence)
    Smax = calculate_makespan(processing_times, best_global_sol)
    for i in range(num_jobs):
        for j in range(i+1, num_jobs):
            new_solution = swap(solution, i, j)
            makespan = calculate_makespan(processing_times, new_solution)

            # First improving solution
            if makespan < Cmax:
                # Improves the global solution
                if makespan < Smax:
                    return new_solution, makespan, new_solution 
                Cmax = makespan
                sequence = new_solution                

    return sequence, Cmax, best_global_sol 

In [15]:
best_global = np.random.permutation(20).tolist()

while calculate_makespan(rnd_data, best_global) > Cmax:
    best_global = np.random.permutation(20).tolist()

neighbor_sol, neighbor_cmax, best_global_found = fba_swap(permutation, rnd_data, best_global)

print("Original_solution: ", permutation)
print("Cmax = ", Cmax)

print("\nBest_original_global_solution: ", best_global)
print("Best_global_initial_cmax = ", calculate_makespan(rnd_data, best_global))

print("\nNeighbor_solution: ", neighbor_sol)
print("Neighor_cmax = ", neighbor_cmax)

print("\nBest_global_solution: ", best_global_found)
print("Best_global_cmax = ", calculate_makespan(rnd_data, best_global_found))

Original_solution:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]
Cmax =  2054

Best_original_global_solution:  [4, 0, 19, 14, 6, 9, 18, 17, 12, 3, 1, 5, 15, 11, 7, 10, 13, 2, 16, 8]
Best_global_initial_cmax =  1914

Neighbor_solution:  [0, 15, 11, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 18, 6, 7, 1, 17, 3, 12]
Neighor_cmax =  1910

Best_global_solution:  [0, 15, 11, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 18, 6, 7, 1, 17, 3, 12]
Best_global_cmax =  1910


## Local Based Metaheuristics

### Simulated annealing (RS) 

#### General method to get neighbors (except first and best admissible)

In [16]:
def get_neighbor(processing_times, solution, method='random_swap'):
    if method == 'random_swap':
        sol, val = random_swap(solution, processing_times)
    elif method == 'best_swap':
        sol, val = best_swap(solution, processing_times)
    elif method == 'first_admissible_swap':
        sol, val = first_admissible_swap(solution, processing_times)
    else:
        i = random.randint(0, 2)
        if i == 0:
            sol, val = random_swap(solution, processing_times)
        elif i == 1:
            sol, val = best_swap(solution, processing_times)
        elif i == 2:
            sol, val = first_admissible_swap(solution, processing_times)
    return sol, val

In [18]:
def RS(processing_times, initial_solution, temp, method='random_swap', alpha=0.6, nb_palier= 10, it_max=100):
    solution = initial_solution.copy()
    makespan = calculate_makespan(processing_times, solution)
    print('init_sol: ',solution, ' makespan = ', makespan, "\n")
    it = 0
    while it < it_max:
        for i in range(nb_palier):
            sol, value = get_neighbor(processing_times, solution, method)
            # print('Swap_sol: ',sol,' makespan = ', value)
            delta = makespan - value
            if delta > 0:
                solution = sol
                makespan = value
            else:
                if random.uniform(0, 1) < math.exp(delta / temp):
                    solution = sol
        temp = alpha * temp
        it += 1
    
    return solution

### RS with first and best admissible swap 

In [19]:
def RS_fba(processing_times, initial_solution, intitial_global, temp, alpha=0.6, nb_palier= 1, it_max=100):
    solution = initial_solution.copy()
    makespan = calculate_makespan(processing_times, solution)
    print('init_sol: ',solution, ' makespan = ', makespan)
    it = 0
    print('initial_global_solution: ',intitial_global, ' global_makespan = ', calculate_makespan(processing_times, intitial_global), "\n")
    best_global = intitial_global.copy()
    while it < it_max :
        for i in range(nb_palier):
            sol, value, best_global = fba_swap(solution, processing_times, best_global)
            # print('FBA_swap_sol: ',sol,' makespan = ', value)
            delta = makespan - value
            if delta > 0:
                solution = sol
                makespan = value
            else:
                if random.uniform(0, 1) < math.exp(delta / temp):
                    solution = sol
        temp = alpha * temp
        it += 1
    
    return solution, best_global

### Paramètres

In [29]:
# Définir l'espace des paramètres
space = [
    Real(0.5, 0.9, name='alpha'),
    Integer(5, 20, name='nb_palier'),
    Integer(50, 200, name='it_max')
]

processing_times = benchmarks[0]
initial_solution = permutation
# Fonction objectif à minimiser
@use_named_args(space)
def objective(**params):
    alpha = params['alpha']
    nb_palier = params['nb_palier']
    it_max = params['it_max']
    # Vous devez définir `processing_times` et `initial_solution`
    result = RS(processing_times, initial_solution, temp=100, alpha=alpha, nb_palier=nb_palier, it_max=it_max)
    return calculate_makespan(processing_times, result)

# Lancer l'optimisation
result = gp_minimize(objective, space, n_calls=50, random_state=0)
# Affichage des meilleurs paramètres trouvés
print("Meilleurs paramètres: alpha={}, nb_palier={}, it_max={}".format(
    result.x[0], result.x[1], result.x[2]))

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12] 

d:\Python\anaconda\lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [0.9, 17, 50] before, using random point [0.8469060216915132, 18, 157]
  warnings.warn(


init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 



d:\Python\anaconda\lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [0.9, 17, 50] before, using random point [0.6986662024605518, 10, 196]
  warnings.warn(


init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 



...

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 



d:\Python\anaconda\lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [0.9, 17, 50] before, using random point [0.8625735151153409, 7, 120]
  warnings.warn(


init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 



d:\Python\anaconda\lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [0.9, 17, 50] before, using random point [0.5231948993445806, 18, 63]
  warnings.warn(


init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 

Meilleurs paramètres: alpha=0.7371378472900074, nb_palier=18, it_max=179


## Tests

We can adjust the swapping method to see the differences in tests

In [24]:
benchmark = benchmarks[0]
optimal_alpha = result.x[0]
optimal_Nb_palier = result.x[1]
optimal__it_max = result.x[2]
best_global = np.random.permutation(20).tolist()
while calculate_makespan(rnd_data, best_global) > Cmax:
    best_global = np.random.permutation(20).tolist()

### Random initial solution

#### Without FBA swapping method

In [25]:
rs_solution = RS(benchmark, permutation, 5, method='random_swap',alpha=optimal_alpha, nb_palier=optimal_Nb_palier, it_max=optimal__it_max)

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444 



#### With FBA swapping method

In [26]:
rs_fba_solution, best_global_found = RS_fba(benchmark, permutation, best_global, 5,alpha=optimal_alpha, nb_palier=optimal_Nb_palier, it_max=optimal__it_max)

init_sol:  [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12]  makespan =  1444
initial_global_solution:  [5, 11, 15, 9, 3, 17, 4, 2, 1, 12, 19, 10, 7, 0, 14, 16, 6, 8, 13, 18]  global_makespan =  1600 



In [27]:
print('Results of random:')
print(f'First sequence: {permutation} with a makespan of {calculate_makespan(benchmark, permutation)}.')
print('\nResults of RS:')
print(f'Best solution: {rs_solution} with a makespan of {calculate_makespan(benchmark, rs_solution)}.')
print('\nResults of RS_FBA:')
print(f'Best solution: {rs_fba_solution} with a makespan of {calculate_makespan(benchmark, rs_fba_solution)}.')
print(f'Best global solution: {best_global_found} with a makespan of {calculate_makespan(benchmark, best_global_found)}.')

Results of random:
First sequence: [0, 15, 18, 4, 16, 10, 2, 5, 19, 9, 8, 14, 13, 11, 6, 7, 1, 17, 3, 12] with a makespan of 1444.

Results of RS:
Best solution: [14, 5, 2, 16, 8, 4, 18, 3, 15, 12, 1, 7, 6, 10, 13, 17, 0, 11, 9, 19] with a makespan of 1297.

Results of RS_FBA:
Best solution: [12, 8, 13, 7, 16, 3, 2, 4, 18, 5, 15, 14, 17, 11, 6, 10, 1, 0, 9, 19] with a makespan of 1297.
Best global solution: [12, 8, 13, 7, 16, 3, 2, 4, 18, 5, 15, 14, 17, 11, 6, 10, 1, 0, 9, 19] with a makespan of 1297.


### NEH initial solution

#### Without FBA swapping method

In [ ]:
initialSolution, makespan = neh_algorithm(benchmark)
rs_solution = RS(benchmark, initialSolution, 5)

init_sol:  [8, 6, 15, 10, 7, 1, 16, 2, 14, 13, 17, 3, 9, 11, 0, 18, 5, 4, 12, 19]  makespan =  1334 



#### With FBA swapping method

In [ ]:
rs_fba_solution, best_global_found = RS_fba(benchmark, initialSolution, best_global, 5)

init_sol:  [8, 6, 15, 10, 7, 1, 16, 2, 14, 13, 17, 3, 9, 11, 0, 18, 5, 4, 12, 19]  makespan =  1334
initial_global_solution:  [12, 2, 15, 7, 10, 6, 17, 0, 19, 1, 18, 9, 8, 4, 13, 5, 11, 14, 16, 3]  global_makespan =  1537 



In [ ]:
print('Results of NEH:')
print(f'First sequence: {initialSolution} with a makespan of {makespan}.')
print('\nResults of RS:')
print(f'Best solution: {rs_solution} with a makespan of {calculate_makespan(benchmark, rs_solution)}.')
print('\nResults of RS_FBA:')
print(f'Best solution: {rs_fba_solution} with a makespan of {calculate_makespan(benchmark, rs_fba_solution)}.')
print(f'Best global solution: {best_global_found} with a makespan of {calculate_makespan(benchmark, best_global_found)}.')

Results of NEH:
First sequence: [8, 6, 15, 10, 7, 1, 16, 2, 14, 13, 17, 3, 9, 11, 0, 18, 5, 4, 12, 19] with a makespan of 1334.

Results of RS:
Best solution: [16, 14, 5, 12, 3, 8, 7, 1, 17, 6, 11, 10, 2, 4, 15, 0, 18, 13, 9, 19] with a makespan of 1297.

Results of RS_FBA:
Best solution: [8, 12, 15, 10, 7, 1, 16, 2, 14, 13, 17, 3, 4, 11, 0, 18, 5, 9, 6, 19] with a makespan of 1305.
Best global solution: [8, 12, 15, 10, 7, 1, 16, 2, 14, 13, 17, 3, 4, 11, 0, 18, 5, 9, 6, 19] with a makespan of 1305.


### PRSKE initial solution

#### Without FBA swapping method

In [ ]:
initialSolution, makespan  = PRSKE(benchmark)
rs_solution = RS(benchmark, initialSolution, 5)

init_sol:  [3, 17, 10, 1, 9, 11, 6, 4, 19, 15, 18, 5, 0, 12, 8, 13, 14, 7, 16, 2]  makespan =  1593 



#### With FBA swapping method

In [ ]:
rs_fba_solution, best_global_found = RS_fba(benchmark, initialSolution, best_global, 5)

init_sol:  [3, 17, 10, 1, 9, 11, 6, 4, 19, 15, 18, 5, 0, 12, 8, 13, 14, 7, 16, 2]  makespan =  1593
initial_global_solution:  [12, 2, 15, 7, 10, 6, 17, 0, 19, 1, 18, 9, 8, 4, 13, 5, 11, 14, 16, 3]  global_makespan =  1537 



In [ ]:
print('Results of PRSKE:')
print(f'First sequence: {initialSolution} with a makespan of {makespan}.')
print('\nResults of RS:')
print(f'Best solution: {rs_solution} with a makespan of {calculate_makespan(benchmark, rs_solution)}')
print('\nResults of RS_FBA:')
print(f'Best solution: {rs_fba_solution} with a makespan of {calculate_makespan(benchmark, rs_fba_solution)}.')
print(f'Best global solution: {best_global_found} with a makespan of {calculate_makespan(benchmark, best_global_found)}.')

Results of PRSKE:
First sequence: [3, 17, 10, 1, 9, 11, 6, 4, 19, 15, 18, 5, 0, 12, 8, 13, 14, 7, 16, 2] with a makespan of 1593.

Results of RS:
Best solution: [14, 1, 5, 0, 8, 3, 13, 10, 4, 6, 16, 2, 17, 18, 7, 11, 15, 12, 9, 19] with a makespan of 1297

Results of RS_FBA:
Best solution: [6, 13, 10, 17, 16, 14, 3, 4, 19, 15, 18, 5, 0, 1, 2, 8, 11, 7, 9, 12] with a makespan of 1359.
Best global solution: [6, 13, 10, 17, 16, 14, 3, 4, 19, 15, 18, 5, 0, 1, 2, 8, 11, 7, 9, 12] with a makespan of 1359.
